<a href="https://colab.research.google.com/github/Seowoo-Jung/ESAA/blob/main/3%EC%A3%BC%EC%B0%A8_%EB%8D%B0%EC%9D%B4%EC%BD%98_%ED%95%84%EC%82%AC_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%B6%84%EC%84%9D(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텍스트 분석(2)

(데이콘 필사)

## 4. Embedding

-  vectorization을 하면 단어/문장 사이의 관계에 대하여 설명할 수 없다. 따라서 embedding이 필요하다.(벡터화는 단어 중요도, 문장 중요도를 구분할 수는 있지만 유사도는 알 수 없다.)

- Embedding의 목적: 비슷한 의미를 내포하고 있는 토큰들은 가깝게, 그렇지 않은 토큰들은 멀리 뿌린다. 

- 이것도 모델링이므로 데이터 훈련이 필요하다.

### 1) Keras Embedding Layer
- 무작위로 특정 차원으로 입력 벡터들을 뿌린 후 학습을 통해 가중치를 조절 --> 단어 사이의 관계를 반영하지 않음.

- 블로그 참고: https://wikidocs.net/32105

- 밀집벡터로 바꿔주어 학습시킴. 처음에 저차원의 랜덤한 숫자들을 갖다가 인공신경망을 통해 유의미한 숫자로 바뀜. 

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

train=pd.read_csv("/content/train.csv")

def text2sequence(train_text, max_len=100):
    
    tokenizer = Tokenizer() #keras의 vectorizing 함수 호출
    tokenizer.fit_on_texts(train_text) #train 문장에 fit
    train_X_seq = tokenizer.texts_to_sequences(train_text) #각 토큰들에 정수 부여
    vocab_size = len(tokenizer.word_index) + 1 #모델에 알려줄 vocabulary의 크기 계산
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len) #설정한 문장의 최대 길이만큼 padding
    
    return X_train, vocab_size, tokenizer

train_X, vocab_size, vectorizer = text2sequence(train['text'], max_len = 100)

vocab_size :  42331


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding

model=Sequential()
model.add(Embedding(vocab_size, 128, input_length=100)) # Embedding(단어개수, 벡터 크기, 입력크기)

### 2) word2vec

- 주변 단어와의 관계를 통해 해당 단어의 의미적 특성을 파악. 주변 맥락을 이용하여 관련된 단어들 유추 할 수 있음. 비지도 학습



In [ ]:
import gensim
word2vec = gensim.models.KeyedVectors.load_word2vec_format("/content/GoogleNews-vectors-negative300.bin.gz", binary = True)

EOFError: ignored

In [ ]:
embedding_matrix=np.zeros((vocab_size, 300)) #300차원의 임베딩 매트릭스 생성

for index, word in enumerate(vectorizer.word_index): # 토큰을 하나씩 넘겨줌
  if word in word2vec: #넘겨 받은 토큰이 word2vec에  존재하면
    embedding_vector=word2vec[word] # 벡터 불러와서
    embedding_matrix[i]=embedding_vector #embedding matrix에 저장
  else:
    print('word2vec에 없는 단어입니다.')
    break

In [ ]:
model=Sequential()
model.add(Embedding(vocav_size, 300, weights = [embedding_matrix], input_length=100))

### 3)glove

- 각 토큰 간의 유사성을 반영하면서 데이터 전체에 대한 빈도도 반영한다 (word2vec의 단점보완)

In [ ]:
# 데이터 불러와서 glove dictionary에 저장하기
glove=dict()
f=open("glove.txt")
for line in f:
  values = line.split()
  word = values[0]
  vector=array(values[1:], dtype='float32')
  glove[word] = vector
f.close()

In [ ]:
# 토큰들을 가져와 embedding matrix에 저장
embedding_matrix=np.zeros((vocav_size, 300))

for index, word in enumerate(vocabulary):
  if word in glove:
    embedding_vector = glove[word]
    embedding_matrix[i] = embedding_vector
  else:
    print('glove 없는 단어입니다.')
    break

In [ ]:
# embedding_matrix를 가중치로 두어야함
model= Sequential()
model.add(Embedding(vocav_size, 300, weights = [embedding_matrix], input_length=100))

### 4) Fasttext

- sub 단어를 단위로 사용한다.




In [ ]:
from gensim.models.keyedvectors import KeyedVectors
FastText = KeyedVectors.load_word2vec_format('./fasttext.bin', binary = True)

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300)) #300차원의 임베딩 매트릭스 생성

for index, word in enumerate(vocabulary): #vocabulary에 있는 토큰들을 하나씩 넘겨줍니다.
    if word in word2vec: #넘겨 받은 토큰이 word2vec에 존재하면(이미 훈련이 된 토큰이라는 뜻)
        embedding_vector = word2vec[word] #해당 토큰에 해당하는 vector를 불러오고
        embedding_mxtrix[i] = embedding_vector #해당 위치의 embedding_mxtrix에 저장합니다.

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 300,weights = [embedding_matrx], input_length = max_len))

# Modeling

5. 대회적용

In [ ]:
def text_preprocessing(text_list):
    
    stopwords = ['을', '를', '이', '가', '은', '는', 'null']
    tokenizer = Okt()
    
    for text in tqdm.tqdm(text_list):
        txt = re.sub('[^가-힣a-z]', ' ', text.lower())
        token = tokenizer.morphs(txt)
        token = [t for t in token if t not in stopwords or type(t) != float]
        
    return token, tokenizer

train['token'], okt = text_preprocessing(train['content'])

- Vectorization

In [ ]:
def text2sequence(train_text, max_len=1000):
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train_text)
    train_X_seq = tokenizer.texts_to_sequences(train_text)
    vocab_size = len(tokenizer.word_index) + 1
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len)
    return X_train, vocab_size, tokenizer

train_y = train['info']
train_X, vocab_size, vectorizer = text2sequence(train['token'], max_len = 100)
print(train_X.shape, train_y.shape)

- Embedding

In [ ]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary = True)
embedding_matrix = np.zeros((vocab_size, 300))

for index, word in enumerate(vocabulary):
    if word in word2vec
        embedding_vector = word2vec[word] 
        embedding_mxtrix[i] = embedding_vector 
    else:
        print("word2vec에 없는 단어입니다.")
        break

- Modeling

In [ ]:
def LSTM(vocab_size, max_len=1000):
    model = Sequential()
    model.add(Embedding(vocab_size, 300,weights = [embedding_matrx], input_length = max_len)) #임베딩 가중치 적용 코드
    model.add(SpatialDropout1D(0.3))
    model.add(LSTM(64))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu', kernel_regularizer = regularizers.l2(0.001)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
    model.summary()
    return model